In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st
from scipy.stats import linregress
import random

In [2]:
#import csv file and create dataframe
csv_path  = "Resources/OSF_socialmedia_data.csv"
social_media_df = pd.read_csv(csv_path)
social_media_df.head()

,Unnamed: 0,Participant,Date,Day,Time,Session.Name,Notification.No,LifePak.Download.No,Responded,Completed.Session,...,Loneliness,Concentrat,LossOfInt,Inferior,Hopeless,Stress,PSMU,AutoPSMU,News,Active
0,1,115091,05/02/2017,Tuesday,10:00:06,Daily questions,1,1,1,1,...,61.0,64.0,68.0,50.0,56.0,73.0,66.0,66,39.0,38.0
1,2,115091,05/02/2017,Tuesday,12:02:48,Daily questions,2,1,1,1,...,57.0,70.0,58.0,37.0,56.0,67.0,0.0,#skipped#,27.0,23.0
2,3,115091,05/02/2017,Tuesday,14:03:02,Daily questions,3,1,1,1,...,44.0,70.0,67.0,26.0,38.0,39.0,63.0,56,34.0,36.0
3,4,115091,05/02/2017,Tuesday,16:34:28,Daily questions,4,1,1,1,...,56.0,79.0,89.0,30.0,20.0,60.0,67.0,68,29.0,23.0
4,5,115091,05/02/2017,Tuesday,18:00:00,Daily questions,5,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
#count to determine missing values
social_media_df.count()


Unnamed: 0                         12245
Participant                        12245
Date                               12245
Day                                12245
Time                               12245
Session.Name                       12245
Notification.No                    12245
LifePak.Download.No                12245
Responded                          12245
Completed.Session                  12245
Session.Instance                    8695
Session.Instance.Response.Lapse     8695
Reminders.Delivered                12245
Instr_DQs                              0
Fatigue                             8653
DeprMood                            8648
Loneliness                          8646
Concentrat                          8645
LossOfInt                           8646
Inferior                            8646
Hopeless                            8650
Stress                              8649
PSMU                                8646
AutoPSMU                            8651
News            

In [4]:
#cleaned up data 
social_media_df.dropna(how = "all")


cleaned_social_media = social_media_df.drop(['LifePak.Download.No','Session.Name', 'Session.Instance.Response.Lapse', 'Instr_DQs', 'Unnamed: 0'], axis = 1)
cleaned_social_media = cleaned_social_media.dropna(axis = 0, how = 'any')
cleaned_social_media.head(10)


cleaned_social_media.to_csv('Resources/cleaned_data.csv')

In [5]:
study_began = cleaned_social_media["Date"].min()
study_began

'03/28/2017'

In [6]:
study_end = cleaned_social_media["Date"].max()
study_end

'05/26/2017'

In [7]:
unique_participants = cleaned_social_media["Participant"].drop_duplicates().count()
unique_participants

125

In [8]:
#remove data before first mass shooting during the timeframe
new_cleaned_social_media = cleaned_social_media.loc[((cleaned_social_media["Date"]> "04/16/2017"))] 
new_cleaned_social_media

,Participant,Date,Day,Time,Notification.No,Responded,Completed.Session,Session.Instance,Reminders.Delivered,Fatigue,...,Loneliness,Concentrat,LossOfInt,Inferior,Hopeless,Stress,PSMU,AutoPSMU,News,Active
0,115091,05/02/2017,Tuesday,10:00:06,1,1,1,1.0,0,61.0,...,61.0,64.0,68.0,50.0,56.0,73.0,66.0,66,39.0,38.0
1,115091,05/02/2017,Tuesday,12:02:48,2,1,1,2.0,0,28.0,...,57.0,70.0,58.0,37.0,56.0,67.0,0.0,#skipped#,27.0,23.0
2,115091,05/02/2017,Tuesday,14:03:02,3,1,1,3.0,0,24.0,...,44.0,70.0,67.0,26.0,38.0,39.0,63.0,56,34.0,36.0
3,115091,05/02/2017,Tuesday,16:34:28,4,1,1,4.0,0,63.0,...,56.0,79.0,89.0,30.0,20.0,60.0,67.0,68,29.0,23.0
5,115091,05/02/2017,Tuesday,20:02:12,6,1,1,5.0,0,16.0,...,0.0,27.0,83.0,15.0,3.0,25.0,16.0,10,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12240,9873359,04/25/2017,Tuesday,14:00:09,3,1,1,81.0,0,15.0,...,0.0,14.0,0.0,0.0,0.0,14.0,22.0,0,0.0,0.0
12241,9873359,04/25/2017,Tuesday,16:40:58,4,1,1,82.0,0,22.0,...,0.0,11.0,0.0,0.0,0.0,0.0,6.0,0,0.0,24.0
12242,9873359,04/25/2017,Tuesday,18:00:06,5,1,1,83.0,0,4.0,...,0.0,10.0,0.0,0.0,0.0,0.0,11.0,0,0.0,18.0
12243,9873359,04/25/2017,Tuesday,20:05:31,6,1,1,84.0,0,0.0,...,8.0,22.0,8.0,0.0,0.0,15.0,10.0,0,0.0,0.0


In [9]:
new_cleaned_social_media["Date"].min()

'04/17/2017'

In [10]:
unique_participants2 = new_cleaned_social_media["Participant"].drop_duplicates().count()
unique_participants2

122

In [11]:
nw_count_df = new_cleaned_social_media.groupby(["Date","Participant"])["News"]
daily_news_stats_df = nw_count_df.describe()
daily_news_stats_df.head()

count       mean        std   min    25%   50%   75%  \
Date       Participant                                                         
04/17/2017 438907         4.0  37.750000  32.622845   0.0  15.75  42.5  64.5   
           572172         6.0  43.666667  20.422210  19.0  26.50  46.0  61.0   
           696084         5.0  42.000000  34.102786   7.0  11.00  38.0  77.0   
           1102830        4.0  26.000000  30.419292   0.0   0.00  23.0  49.0   
           1274514        6.0  24.500000  17.896927   0.0  13.75  24.5  34.5   

                         max  
Date       Participant        
04/17/2017 438907       66.0  
           572172       65.0  
           696084       77.0  
           1102830      58.0  
           1274514      50.0

In [12]:
max_news_entry = daily_news_stats_df["count"].max()
max_news_entry

10.0

In [13]:
drop_newssurvey_entries = daily_news_stats_df.loc[((daily_news_stats_df["count"]>4))] 
drop_newssurvey_entries

count       mean        std   min    25%   50%   75%  \
Date       Participant                                                         
04/17/2017 572172         6.0  43.666667  20.422210  19.0  26.50  46.0  61.0   
           696084         5.0  42.000000  34.102786   7.0  11.00  38.0  77.0   
           1274514        6.0  24.500000  17.896927   0.0  13.75  24.5  34.5   
           1318587        5.0  10.400000  16.577093   0.0   0.00   0.0  14.0   
           1500743        6.0  27.500000  23.805462   0.0  10.75  24.5  40.5   
...                       ...        ...        ...   ...    ...   ...   ...   
05/24/2017 5625691        5.0  41.000000  38.457769   0.0   0.00  54.0  74.0   
           7949149        6.0  57.333333   3.141125  53.0  55.00  58.5  59.0   
           8353768        5.0   7.200000   8.843076   0.0   1.00   3.0  11.0   
           8810136        5.0  30.400000  22.611944   0.0  22.00  26.0  45.0   
           9464374        5.0  12.600000  17.544230   0.0   0.00   0.0  27.0   

                         max  
Date       Participant        
04/17/2017 572172       65.0  
           696084       77.0  
           1274514      50.0  
           1318587      38.0  
           1500743      64.0  
...                      ...  
05/24/2017 5625691      77.0  
           7949149      61.0  
           8353768      21.0  
           8810136      59.0  
           9464374      36.0  

[794 rows x 8 columns]

In [14]:
average_rate_news_entry = drop_newssurvey_entries.loc[((drop_newssurvey_entries["min"]>49.9))] 
average_rate_news_entry.head(50)

count       mean       std   min    25%   50%    75%  \
Date       Participant                                                         
04/17/2017 4188405        7.0  66.571429  4.894117  57.0  66.00  67.0  68.50   
           4850438        5.0  50.000000  0.000000  50.0  50.00  50.0  50.00   
04/19/2017 4188405        7.0  68.000000  5.916080  62.0  63.50  67.0  72.00   
           6716111        7.0  76.714286  4.644505  71.0  74.00  76.0  79.00   
04/20/2017 4188405        7.0  59.571429  2.819997  57.0  57.50  59.0  60.50   
           6716111        5.0  85.400000  7.469940  76.0  82.00  83.0  92.00   
04/21/2017 6716111        5.0  81.800000  5.167204  74.0  81.00  81.0  86.00   
04/22/2017 4188405        7.0  64.142857  1.676163  63.0  63.00  63.0  65.00   
04/24/2017 5663344        5.0  66.200000  7.791020  55.0  63.00  68.0  69.00   
04/25/2017 4188405        7.0  62.714286  1.253566  61.0  62.00  62.0  64.00   
04/26/2017 4188405        6.0  62.000000  4.289522  56.0  59.50  62.0  64.50   
05/01/2017 3234198        6.0  71.333333  6.314006  64.0  67.50  70.5  73.50   
05/03/2017 4623292        6.0  66.500000  7.395945  60.0  63.25  64.5  65.75   
05/04/2017 8560321        5.0  78.600000  9.607289  64.0  76.00  79.0  85.00   
05/05/2017 857523         7.0  67.428571  3.505098  64.0  65.00  67.0  68.50   
           7706886        5.0  55.600000  6.767570  51.0  51.00  51.0  59.00   
05/07/2017 857523         6.0  71.500000  5.049752  67.0  67.00  70.5  76.25   
05/09/2017 857523         7.0  74.714286  2.927700  69.0  74.00  75.0  77.00   
05/10/2017 7345751        5.0  71.000000  5.873670  65.0  67.00  70.0  73.00   
05/11/2017 857523         7.0  71.428571  4.157609  64.0  70.00  72.0  73.50   
05/12/2017 857523         6.0  73.500000  4.415880  67.0  72.00  72.5  76.75   
05/13/2017 857523         6.0  73.500000  4.324350  69.0  70.50  73.0  74.75   
           8560321        7.0  55.571429  6.451283  50.0  50.50  55.0  57.50   
05/14/2017 4623292        7.0  65.857143  5.367450  57.0  63.00  68.0  68.50   
05/16/2017 7706886        5.0  70.800000  4.969909  66.0  68.00  70.0  71.00   
05/17/2017 8560321        5.0  62.600000  5.983310  56.0  60.00  61.0  64.00   
05/18/2017 857523         6.0  74.666667  2.065591  72.0  73.25  74.5  76.50   
           883656         6.0  85.000000  5.440588  79.0  81.00  85.0  86.75   
05/20/2017 883656         6.0  77.666667  5.046451  70.0  75.50  78.0  79.75   
05/23/2017 883656         6.0  84.000000  9.818350  75.0  76.25  82.5  88.00   
05/24/2017 7949149        6.0  57.333333  3.141125  53.0  55.00  58.5  59.00   

                          max  
Date       Participant         
04/17/2017 4188405       73.0  
           4850438       50.0  
04/19/2017 4188405       76.0  
           6716111       84.0  
04/20/2017 4188405       65.0  
           6716111       94.0  
04/21/2017 6716111       87.0  
04/22/2017 4188405       67.0  
04/24/2017 5663344       76.0  
04/25/2017 4188405       64.0  
04/26/2017 4188405       68.0  
05/01/2017 3234198       82.0  
05/03/2017 4623292       81.0  
05/04/2017 8560321       89.0  
05/05/2017 857523        74.0  
           7706886       66.0  
05/07/2017 857523        77.0  
05/09/2017 857523        77.0  
05/10/2017 7345751       80.0  
05/11/2017 857523        77.0  
05/12/2017 857523        79.0  
05/13/2017 857523        81.0  
           8560321       68.0  
05/14/2017 4623292       73.0  
05/16/2017 7706886       79.0  
05/17/2017 8560321       72.0  
05/18/2017 857523        77.0  
           883656        94.0  
05/20/2017 883656        85.0  
05/23/2017 883656       100.0  
05/24/2017 7949149       61.0